In [2]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import sys 
import os

sys.path.append(os.path.abspath('..')) 

from utils.preproc_utils import *
from utils.plot_utils import *
from utils.modeling_utils import *

%load_ext autoreload
%autoreload 2


Importing clusters

In [10]:
sys.path.append(os.path.abspath('..')) 
base_dir = os.path.abspath(os.path.join(os.getcwd(), '../../data/clusters/'))

parents = pd.read_csv(base_dir + '/cluster_1.0.csv')
promo_hunters= pd.read_csv(base_dir + '/cluster_2.0.csv')
veggies= pd.read_csv(base_dir + '/cluster_3.0.csv')
high_spenders = pd.read_csv(base_dir + '/cluster_4.0.csv')
loyal_customers = pd.read_csv(base_dir + '/cluster_5.0.csv')
young_drinkers = pd.read_csv(base_dir + '/cluster_6.0.csv')
young_tech_gamers = pd.read_csv(base_dir + '/cluster_7.0.csv')
pet_lovers = pd.read_csv(base_dir + '/cluster_0.0.csv')
fishies = pd.read_csv(base_dir + '/fishy_outliers.csv')

basket_path = os.path.abspath(os.path.join(os.getcwd(), '../../data/raw/'))


basket = pd.read_csv(basket_path + '/customer_basket.csv')

In [11]:
for i in range(1, 9):
    print(eval(f"cluster_{i}").size)


163116
249136
163070
129766
113873
25438
52647
97543


In [23]:
def association_rules_pipeline(cluster, basket, join_column='customer_id', list_column='list_of_goods',
                              min_support=0.2, metric='lift', min_threshold=1):
    """
    Perform the association rules pipeline on customer-basket data.

    Args:
        cluster (pandas.DataFrame): The cluster data.
        basket (pandas.DataFrame): The basket data.
        join_column (str, optional): The column name used for joining the cluster and basket data. Defaults to 'customer_id'.
        list_column (str, optional): The column name containing the list of goods in the basket. Defaults to 'list_of_goods'.
        min_support (float, optional): The minimum support threshold for generating frequent itemsets. Defaults to 0.2.
        metric (str, optional): The metric used for evaluating association rules. Defaults to 'lift'.
        min_threshold (float, optional): The minimum threshold for the metric to consider a rule. Defaults to 1.

    Returns:
        pandas.DataFrame: The generated association rules.
    """
    # Filter basket data based on customer_id in cluster
    basket_filtered = basket[basket[join_column].isin(cluster[join_column])]

    # Extract transactions from the filtered basket data
    transactions = basket_filtered[list_column].apply(lambda x: [item.strip() for item in x[1:-1].split(',')])

    # Convert transactions to transaction matrix using TransactionEncoder
    te = TransactionEncoder()
    te_fit = te.fit(transactions).transform(transactions)
    transactions_items = pd.DataFrame(te_fit, columns=te.columns_)

    # Generate frequent itemsets using Apriori algorithm
    frequent_itemsets = apriori(transactions_items, min_support=min_support, use_colnames=True)

    # Check if frequent_itemsets is empty
    if frequent_itemsets.empty:
        print("No frequent itemsets found.")
        return pd.DataFrame()

    # Generate association rules from frequent itemsets
    rules = association_rules(frequent_itemsets, metric=metric, min_threshold=min_threshold)

    return rules

In [12]:
clusters = [pet_lovers,high_spenders,young_drinkers,young_tech_gamers,fishies,veggies,promo_hunters,loyal_customers,parents]  # replace with your actual cluster DataFrames


In [25]:
clusters = {'gamers': young_tech_gamers, 
            'spenders': high_spenders, 
            'veggies': veggies, 
            'promo_hunters': promo_hunters, 
            'alcoholics': young_drinkers, 
            'parents': parents, 
            'young_tech_gamers': young_tech_gamers, 
            'pet lovers': pet_lovers, 
            'loyal customers': loyal_customers, 
            'fishies': fishies
            }

# Display the association rules for each cluster
# The association rules are sorted by lift in descending order
# Only the top 60 association rules are displayed
for cluster_name, cluster in clusters.items():
    print(cluster_name.upper())
    result = association_rules_pipeline(cluster, basket)
    
    if not result.empty and 'lift' in result.columns:
        display(result.sort_values('lift', ascending=False).head(60))
    else:
        print("No association rules with 'lift' metric found for this cluster.")


GAMERS


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
21,('cake'),"('cooking oil', 'oil')",0.436650,0.473695,0.229754,0.526173,1.110786,0.022915,1.110755,0.177042
16,"('cooking oil', 'oil')",('cake'),0.473695,0.436650,0.229754,0.485025,1.110786,0.022915,1.093936,0.189503
19,('cooking oil'),"('oil', 'cake')",0.565911,0.365714,0.229754,0.405989,1.110126,0.022792,1.067801,0.228527
18,"('oil', 'cake')",('cooking oil'),0.365714,0.565911,0.229754,0.628233,1.110126,0.022792,1.167636,0.156398
4,('candy bars'),('cooking oil'),0.345616,0.565911,0.214187,0.619726,1.095094,0.018599,1.141517,0.132700
5,('cooking oil'),('candy bars'),0.565911,0.345616,0.214187,0.378482,1.095094,0.018599,1.052880,0.200044
20,('oil'),"('cooking oil', 'cake')",0.797438,0.264828,0.229754,0.288115,1.087933,0.018570,1.032712,0.399018
17,"('cooking oil', 'cake')",('oil'),0.264828,0.797438,0.229754,0.867560,1.087933,0.018570,1.529454,0.109941
1,('cake'),('cooking oil'),0.436650,0.565911,0.264828,0.606498,1.071719,0.017722,1.103143,0.118789
0,('cooking oil'),('cake'),0.565911,0.436650,0.264828,0.467967,1.071719,0.017722,1.058862,0.154162


SPENDERS


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
6,('champagne'),('laptop'),0.737687,0.258530,0.203721,0.276162,1.068199,0.013006,1.024358,0.243391
7,('laptop'),('champagne'),0.258530,0.737687,0.203721,0.787996,1.068199,0.013006,1.237304,0.086105
4,('bluetooth headphones'),('samsung galaxy 10'),0.388683,0.506948,0.210128,0.540616,1.066413,0.013086,1.073289,0.101874
5,('samsung galaxy 10'),('bluetooth headphones'),0.506948,0.388683,0.210128,0.414497,1.066413,0.013086,1.044088,0.126310
2,('bluetooth headphones'),('champagne'),0.388683,0.737687,0.304925,0.784508,1.063470,0.018199,1.217276,0.097629
3,('champagne'),('bluetooth headphones'),0.737687,0.388683,0.304925,0.413353,1.063470,0.018199,1.042052,0.227523
10,('champagne'),('spaghetti'),0.737687,0.303690,0.236992,0.321264,1.057869,0.012964,1.025892,0.208542
11,('spaghetti'),('champagne'),0.303690,0.737687,0.236992,0.780376,1.057869,0.012964,1.194373,0.078561
9,('samsung galaxy 10'),('champagne'),0.506948,0.737687,0.395553,0.780265,1.057718,0.021585,1.193769,0.110675
8,('champagne'),('samsung galaxy 10'),0.737687,0.506948,0.395553,0.536208,1.057718,0.021585,1.063088,0.208028


VEGGIES


...

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
21,('cake'),"('cooking oil', 'oil')",0.436650,0.473695,0.229754,0.526173,1.110786,0.022915,1.110755,0.177042
16,"('cooking oil', 'oil')",('cake'),0.473695,0.436650,0.229754,0.485025,1.110786,0.022915,1.093936,0.189503
19,('cooking oil'),"('oil', 'cake')",0.565911,0.365714,0.229754,0.405989,1.110126,0.022792,1.067801,0.228527
18,"('oil', 'cake')",('cooking oil'),0.365714,0.565911,0.229754,0.628233,1.110126,0.022792,1.167636,0.156398
4,('candy bars'),('cooking oil'),0.345616,0.565911,0.214187,0.619726,1.095094,0.018599,1.141517,0.132700
5,('cooking oil'),('candy bars'),0.565911,0.345616,0.214187,0.378482,1.095094,0.018599,1.052880,0.200044
20,('oil'),"('cooking oil', 'cake')",0.797438,0.264828,0.229754,0.288115,1.087933,0.018570,1.032712,0.399018
17,"('cooking oil', 'cake')",('oil'),0.264828,0.797438,0.229754,0.867560,1.087933,0.018570,1.529454,0.109941
1,('cake'),('cooking oil'),0.436650,0.565911,0.264828,0.606498,1.071719,0.017722,1.103143,0.118789
0,('cooking oil'),('cake'),0.565911,0.436650,0.264828,0.467967,1.071719,0.017722,1.058862,0.154162


PET LOVERS
No frequent itemsets found.
No association rules with 'lift' metric found for this cluster.
LOYAL CUSTOMERS


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
19,('cake'),"('cooking oil', 'oil')",0.442840,0.473302,0.230217,0.519866,1.098381,0.020620,1.096981,0.160760
14,"('cooking oil', 'oil')",('cake'),0.473302,0.442840,0.230217,0.486407,1.098381,0.020620,1.084828,0.170057
17,('cooking oil'),"('oil', 'cake')",0.570378,0.371061,0.230217,0.403622,1.087751,0.018572,1.054598,0.187774
16,"('oil', 'cake')",('cooking oil'),0.371061,0.570378,0.230217,0.620429,1.087751,0.018572,1.131863,0.128267
18,('oil'),"('cooking oil', 'cake')",0.797094,0.267857,0.230217,0.288821,1.078263,0.016710,1.029477,0.357716
15,"('cooking oil', 'cake')",('oil'),0.267857,0.797094,0.230217,0.859477,1.078263,0.016710,1.443937,0.099138
13,('muffins'),('oil'),0.247987,0.797094,0.210697,0.849629,1.065909,0.013028,1.349374,0.082224
12,('oil'),('muffins'),0.797094,0.247987,0.210697,0.264331,1.065909,0.013028,1.022217,0.304739
1,('cake'),('cooking oil'),0.442840,0.570378,0.267857,0.604863,1.060459,0.015271,1.087272,0.102326
0,('cooking oil'),('cake'),0.570378,0.442840,0.267857,0.469613,1.060459,0.015271,1.050479,0.132703


FISHIES


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
16,('salmon'),"('fresh tuna', 'shrimp')",0.426843,0.443344,0.206821,0.484536,1.092911,0.017582,1.079912,0.148324
13,"('fresh tuna', 'shrimp')",('salmon'),0.443344,0.426843,0.206821,0.466501,1.092911,0.017582,1.074337,0.152721
12,"('salmon', 'shrimp')",('fresh tuna'),0.342134,0.553355,0.206821,0.604502,1.092429,0.017499,1.129321,0.128611
17,('fresh tuna'),"('salmon', 'shrimp')",0.553355,0.342134,0.206821,0.373757,1.092429,0.017499,1.050497,0.189432
4,('salmon'),('fresh tuna'),0.426843,0.553355,0.255226,0.597938,1.080568,0.019030,1.110885,0.130088
5,('fresh tuna'),('salmon'),0.553355,0.426843,0.255226,0.461233,1.080568,0.019030,1.063831,0.166936
2,('catfish'),('shrimp'),0.243124,0.778878,0.201320,0.828054,1.063138,0.011956,1.286000,0.078465
3,('shrimp'),('catfish'),0.778878,0.243124,0.201320,0.258475,1.063138,0.011956,1.020701,0.268575
1,('shrimp'),('canned_tuna'),0.778878,0.320132,0.264026,0.338983,1.058885,0.014683,1.028518,0.251493
0,('canned_tuna'),('shrimp'),0.320132,0.778878,0.264026,0.824742,1.058885,0.014683,1.261697,0.081796


In [ ]:
clusters = {'gamers': young_tech_gamers, 
            'spenders': high_spenders, 
            'veggies': veggies, 
            'promo_hunters': promo_hunters, 
            'alcoholics': young_drinkers, 
            'parents': parents, 
            'young_tech_gamers': young_tech_gamers, 
            'pet lovers': pet_lovers, 
            'loyal customers': loyal_customers, 
            'fishies': fishies
            }

# Display the association rules for each cluster
# The association rules are sorted by lift in descending order
# Only the top 60 association rules are displayed
for cluster_name, cluster in clusters.items():
    print(cluster_name.upper())
    result = association_rules_pipeline(cluster, basket)
    
    if not result.empty and 'lift' in result.columns:
        display(result.sort_values('lift', ascending=False).head(60))
    else:
        print("No association rules with 'lift' metric found for this cluster.")


GAMERS


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
21,('cake'),"('cooking oil', 'oil')",0.436650,0.473695,0.229754,0.526173,1.110786,0.022915,1.110755,0.177042
16,"('cooking oil', 'oil')",('cake'),0.473695,0.436650,0.229754,0.485025,1.110786,0.022915,1.093936,0.189503
19,('cooking oil'),"('oil', 'cake')",0.565911,0.365714,0.229754,0.405989,1.110126,0.022792,1.067801,0.228527
18,"('oil', 'cake')",('cooking oil'),0.365714,0.565911,0.229754,0.628233,1.110126,0.022792,1.167636,0.156398
4,('candy bars'),('cooking oil'),0.345616,0.565911,0.214187,0.619726,1.095094,0.018599,1.141517,0.132700
5,('cooking oil'),('candy bars'),0.565911,0.345616,0.214187,0.378482,1.095094,0.018599,1.052880,0.200044
20,('oil'),"('cooking oil', 'cake')",0.797438,0.264828,0.229754,0.288115,1.087933,0.018570,1.032712,0.399018
17,"('cooking oil', 'cake')",('oil'),0.264828,0.797438,0.229754,0.867560,1.087933,0.018570,1.529454,0.109941
1,('cake'),('cooking oil'),0.436650,0.565911,0.264828,0.606498,1.071719,0.017722,1.103143,0.118789
0,('cooking oil'),('cake'),0.565911,0.436650,0.264828,0.467967,1.071719,0.017722,1.058862,0.154162


SPENDERS


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
6,('champagne'),('laptop'),0.737687,0.258530,0.203721,0.276162,1.068199,0.013006,1.024358,0.243391
7,('laptop'),('champagne'),0.258530,0.737687,0.203721,0.787996,1.068199,0.013006,1.237304,0.086105
4,('bluetooth headphones'),('samsung galaxy 10'),0.388683,0.506948,0.210128,0.540616,1.066413,0.013086,1.073289,0.101874
5,('samsung galaxy 10'),('bluetooth headphones'),0.506948,0.388683,0.210128,0.414497,1.066413,0.013086,1.044088,0.126310
2,('bluetooth headphones'),('champagne'),0.388683,0.737687,0.304925,0.784508,1.063470,0.018199,1.217276,0.097629
3,('champagne'),('bluetooth headphones'),0.737687,0.388683,0.304925,0.413353,1.063470,0.018199,1.042052,0.227523
10,('champagne'),('spaghetti'),0.737687,0.303690,0.236992,0.321264,1.057869,0.012964,1.025892,0.208542
11,('spaghetti'),('champagne'),0.303690,0.737687,0.236992,0.780376,1.057869,0.012964,1.194373,0.078561
9,('samsung galaxy 10'),('champagne'),0.506948,0.737687,0.395553,0.780265,1.057718,0.021585,1.193769,0.110675
8,('champagne'),('samsung galaxy 10'),0.737687,0.506948,0.395553,0.536208,1.057718,0.021585,1.063088,0.208028


VEGGIES


...

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
21,('cake'),"('cooking oil', 'oil')",0.436650,0.473695,0.229754,0.526173,1.110786,0.022915,1.110755,0.177042
16,"('cooking oil', 'oil')",('cake'),0.473695,0.436650,0.229754,0.485025,1.110786,0.022915,1.093936,0.189503
19,('cooking oil'),"('oil', 'cake')",0.565911,0.365714,0.229754,0.405989,1.110126,0.022792,1.067801,0.228527
18,"('oil', 'cake')",('cooking oil'),0.365714,0.565911,0.229754,0.628233,1.110126,0.022792,1.167636,0.156398
4,('candy bars'),('cooking oil'),0.345616,0.565911,0.214187,0.619726,1.095094,0.018599,1.141517,0.132700
5,('cooking oil'),('candy bars'),0.565911,0.345616,0.214187,0.378482,1.095094,0.018599,1.052880,0.200044
20,('oil'),"('cooking oil', 'cake')",0.797438,0.264828,0.229754,0.288115,1.087933,0.018570,1.032712,0.399018
17,"('cooking oil', 'cake')",('oil'),0.264828,0.797438,0.229754,0.867560,1.087933,0.018570,1.529454,0.109941
1,('cake'),('cooking oil'),0.436650,0.565911,0.264828,0.606498,1.071719,0.017722,1.103143,0.118789
0,('cooking oil'),('cake'),0.565911,0.436650,0.264828,0.467967,1.071719,0.017722,1.058862,0.154162


PET LOVERS
No frequent itemsets found.
No association rules with 'lift' metric found for this cluster.
LOYAL CUSTOMERS


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
19,('cake'),"('cooking oil', 'oil')",0.442840,0.473302,0.230217,0.519866,1.098381,0.020620,1.096981,0.160760
14,"('cooking oil', 'oil')",('cake'),0.473302,0.442840,0.230217,0.486407,1.098381,0.020620,1.084828,0.170057
17,('cooking oil'),"('oil', 'cake')",0.570378,0.371061,0.230217,0.403622,1.087751,0.018572,1.054598,0.187774
16,"('oil', 'cake')",('cooking oil'),0.371061,0.570378,0.230217,0.620429,1.087751,0.018572,1.131863,0.128267
18,('oil'),"('cooking oil', 'cake')",0.797094,0.267857,0.230217,0.288821,1.078263,0.016710,1.029477,0.357716
15,"('cooking oil', 'cake')",('oil'),0.267857,0.797094,0.230217,0.859477,1.078263,0.016710,1.443937,0.099138
13,('muffins'),('oil'),0.247987,0.797094,0.210697,0.849629,1.065909,0.013028,1.349374,0.082224
12,('oil'),('muffins'),0.797094,0.247987,0.210697,0.264331,1.065909,0.013028,1.022217,0.304739
1,('cake'),('cooking oil'),0.442840,0.570378,0.267857,0.604863,1.060459,0.015271,1.087272,0.102326
0,('cooking oil'),('cake'),0.570378,0.442840,0.267857,0.469613,1.060459,0.015271,1.050479,0.132703


FISHIES


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
16,('salmon'),"('fresh tuna', 'shrimp')",0.426843,0.443344,0.206821,0.484536,1.092911,0.017582,1.079912,0.148324
13,"('fresh tuna', 'shrimp')",('salmon'),0.443344,0.426843,0.206821,0.466501,1.092911,0.017582,1.074337,0.152721
12,"('salmon', 'shrimp')",('fresh tuna'),0.342134,0.553355,0.206821,0.604502,1.092429,0.017499,1.129321,0.128611
17,('fresh tuna'),"('salmon', 'shrimp')",0.553355,0.342134,0.206821,0.373757,1.092429,0.017499,1.050497,0.189432
4,('salmon'),('fresh tuna'),0.426843,0.553355,0.255226,0.597938,1.080568,0.019030,1.110885,0.130088
5,('fresh tuna'),('salmon'),0.553355,0.426843,0.255226,0.461233,1.080568,0.019030,1.063831,0.166936
2,('catfish'),('shrimp'),0.243124,0.778878,0.201320,0.828054,1.063138,0.011956,1.286000,0.078465
3,('shrimp'),('catfish'),0.778878,0.243124,0.201320,0.258475,1.063138,0.011956,1.020701,0.268575
1,('shrimp'),('canned_tuna'),0.778878,0.320132,0.264026,0.338983,1.058885,0.014683,1.028518,0.251493
0,('canned_tuna'),('shrimp'),0.320132,0.778878,0.264026,0.824742,1.058885,0.014683,1.261697,0.081796
